In [84]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import StratifiedKFold as SK
from sklearn.model_selection import train_test_split

In [154]:
titanic = pd.read_csv('files/titanic.csv')
data = titanic.copy() 
# data[['Age', 'Fare']] = data[['Age', 'Fare']].fillna(data[['Age', 'Fare']].mean())

y = data['Survived']
X = data.drop(columns=['Survived'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
np.sum(data['Age'].isnull())

177

In [144]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
import xgboost
import catboost

categorical = ['Sex', 'Embarked']
numeric_features = ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']
column_transformer = ColumnTransformer([
    ('ohe', OrdinalEncoder(), categorical),
    ('scaling', StandardScaler(), numeric_features) # how to not scaling?
])

pipeline0 = Pipeline(steps=[
    ('preprocessor', column_transformer),
    ('regression', HistGradientBoostingClassifier(max_depth=3, random_state=42, learning_rate=1e-1))
])


pipeline1 = Pipeline(steps=[
    ('preprocessor', column_transformer),
    ('regression', catboost.CatBoostClassifier(max_depth=3, n_estimators=100, random_state=41, learning_rate=1e-1))
])

pipeline2 = Pipeline(steps=[
    ('preprocessor', column_transformer),
    ('regression', xgboost.XGBClassifier(max_depth=3, eval_metric='logloss', n_estimators=100, random_state=40, learning_rate=1e-1, use_label_encoder=True))
])
# model = pipeline.fit(X_train, y_train)
# y_predict = model.predict(X_test)
# accuracy_score(y_predict, y_test)

In [145]:
model = VotingClassifier(estimators=[
        ('p0', pipeline0), ('p1', pipeline0), ('p2', pipeline0)], voting='hard').fit(X_train, y_train)

In [150]:
model = pipeline1.fit(X_train, y_train)

0:	learn: 0.6552874	total: 559us	remaining: 55.4ms
1:	learn: 0.6240613	total: 1.07ms	remaining: 52.3ms
2:	learn: 0.5973031	total: 1.65ms	remaining: 53.5ms
3:	learn: 0.5749420	total: 2.07ms	remaining: 49.8ms
4:	learn: 0.5564055	total: 2.48ms	remaining: 47.2ms
5:	learn: 0.5398172	total: 2.91ms	remaining: 45.7ms
6:	learn: 0.5261048	total: 3.35ms	remaining: 44.5ms
7:	learn: 0.5161985	total: 3.79ms	remaining: 43.5ms
8:	learn: 0.5058830	total: 4.19ms	remaining: 42.4ms
9:	learn: 0.4977429	total: 4.63ms	remaining: 41.6ms
10:	learn: 0.4914926	total: 5.17ms	remaining: 41.9ms
11:	learn: 0.4845303	total: 5.6ms	remaining: 41.1ms
12:	learn: 0.4818997	total: 6.02ms	remaining: 40.3ms
13:	learn: 0.4769520	total: 6.53ms	remaining: 40.1ms
14:	learn: 0.4733918	total: 7ms	remaining: 39.7ms
15:	learn: 0.4685650	total: 7.5ms	remaining: 39.4ms
16:	learn: 0.4651845	total: 7.91ms	remaining: 38.6ms
17:	learn: 0.4614386	total: 8.37ms	remaining: 38.1ms
18:	learn: 0.4575054	total: 8.78ms	remaining: 37.4ms
19:	learn

In [151]:
y_predict = model.predict(X_test)
accuracy_score(y_predict, y_test)

0.7932960893854749

In [147]:
X_testT = pd.read_csv('files/titanic_test.csv')
X_testT[['Age', 'Fare']] = X_testT[['Age', 'Fare']].fillna(data[['Age', 'Fare']].mean())
y_pred = model.predict(X_testT)

answer = pd.DataFrame({'PassengerId': X_testT.PassengerId, 'Survived': y_pred})
answer.to_csv('files/titanic_answ.csv', index=False)